In [1]:
import sys
sys.path.append('../saerch')
import family
import json
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matrix
import plotly.io as pio
%load_ext autoreload
%autoreload 2
import numpy as np
import json
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import torch

# feature family heatmap

In [74]:
mode = 'epoch_astroPH_old'
SAE_DATA_DIR = '../saerch/sae_data_{}/'.format('astroPH')
DATA_DIR = '../data/vector_store_astroPH/'

abstract_embeddings = np.load(DATA_DIR + "/abstract_embeddings.npy")
abstract_embeddings = abstract_embeddings.astype(np.float32)
abstract_texts = json.load(open(DATA_DIR + '/abstract_texts.json'))['abstracts']

dataset = TensorDataset(torch.from_numpy(abstract_embeddings))
dataloader = DataLoader(dataset, batch_size=1024, shuffle=False)
num_abstracts = len(abstract_embeddings)

model_16_a = family.Model(sae_data_dir = SAE_DATA_DIR, model_path = '../models/16_3072_24_auxk_{}_50.pth'.format(mode), autointerp_results = 'feature_analysis_results_16.json',
                          dataloader = dataloader, num_abstracts = num_abstracts)
model_32_a = family.Model(sae_data_dir = SAE_DATA_DIR, model_path = '../models/32_6144_48_auxk_{}_50.pth'.format(mode),  autointerp_results = 'feature_analysis_results_32.json',
                          dataloader = dataloader, num_abstracts = num_abstracts)
model_64_a = family.Model(sae_data_dir = SAE_DATA_DIR, model_path = '../models/64_9216_128_auxk_{}_50.pth'.format(mode),autointerp_results = 'feature_analysis_results_64.json',
                          dataloader = dataloader, num_abstracts = num_abstracts)

271492 3072
271492 3072
271492 6144
271492 6144
271492 9216
271492 9216


In [75]:
families_64_a = model_64_a.load_all_families()

In [5]:
mode = 'csLG_epoch'
SAE_DATA_DIR = '../saerch/sae_data_{}/'.format('csLG')
DATA_DIR = '../data/vector_store_csLG/'

abstract_embeddings = np.load(DATA_DIR + "/abstract_embeddings.npy")
abstract_embeddings = abstract_embeddings.astype(np.float32)
abstract_texts = json.load(open(DATA_DIR + '/abstract_texts.json'))['abstracts']

dataset = TensorDataset(torch.from_numpy(abstract_embeddings))
dataloader = DataLoader(dataset, batch_size=1024, shuffle=False)
num_abstracts = len(abstract_embeddings)

model_16_c = family.Model(sae_data_dir = SAE_DATA_DIR, model_path = '../models/16_3072_32_auxk_{}_88.pth'.format(mode), 
                       autointerp_results = 'feature_analysis_results_16.json', dataloader = dataloader, num_abstracts = num_abstracts)

model_32_c = family.Model(sae_data_dir = SAE_DATA_DIR, model_path = '../models/32_6144_64_auxk_{}_88.pth'.format(mode),
                        autointerp_results = 'feature_analysis_results_32.json', dataloader = dataloader, num_abstracts = num_abstracts)

model_64_c = family.Model(sae_data_dir = SAE_DATA_DIR, model_path = '../models/64_9216_128_auxk_{}_88.pth'.format(mode), 
                           autointerp_results = 'feature_analysis_results_64.json', dataloader = dataloader, num_abstracts = num_abstracts)

153146 3072
153146 3072
153146 6144
153146 6144
153146 9216
153146 9216


In [7]:
families_64_c = model_64_c.load_all_families()

In [6]:
def make_single_heatmap(parent, model, display = 'cooc'):
    all_names, ids = parent.get_names_and_ids(model)
    
    norm_mat, ratio = parent.matrix(model, mode = display, return_mat = True)

    np.fill_diagonal(norm_mat, 1)

    heatmap = go.Heatmap(
        z=norm_mat,
        x=all_names,
        y=all_names,
        hoverongaps=False,
        hovertemplate='Name: %{x} <br> %{y} <br> %{z}'
    )

    # Create the figure
    fig = go.Figure(data=[heatmap])

    # Update layout for better readability and show only parent labels
    fig.update_layout(
        width=680,
        height=600,
        yaxis = dict(scaleanchor = 'x'),
        plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=20, r=20, t=20, b=20),
    )
    
    fig.show()
    

In [200]:
fam = families_64_c['Gradient Descent Variants and Applications']
mat, val = fam.matrix(return_mat = True, model = model_64_c)
mat[mat < 0.07] = 0
mat[mat < 0.07] = 0
mat = np.round(mat, 2)

clean_labels_subset = {x[0]: model_64_c.clean_labels[x[0]] for x in fam.children}
clean_labels_subset[fam.parent[0]] = model_64_c.clean_labels[fam.parent[0]]

In [201]:
matrix.make_graph(mat, clean_labels_subset, filename = 'learning.graphml')

In [194]:
families_64_c['Gradient Descent Variants and Applications'].children
# Emotion recognition and analysis

[["Newton's Method in Optimization", 700],
 ['Stochastic Variance Reduced Gradient (SVRG)', 934],
 ['Step Size Adaptation in Optimization', 998],
 ['Preconditioning Techniques', 1083],
 ['Adam and RMSProp optimization algorithms', 1460],
 ['Dual Coordinate Ascent in ML', 2243],
 ['Coordinate Descent and Minimax Optimization', 2560],
 ['Escaping saddle points in optimization', 3381],
 ['Adam optimization algorithm', 3535],
 ['Mirror Descent and its applications', 4262],
 ['Learning rate in neural networks', 4647],
 ['SAGA algorithm and its variants', 5039],
 ['Momentum in optimization algorithms', 5176],
 ['Coordinate Descent Optimization Techniques', 5820],
 ['Polyak step size in optimization', 6031],
 ['Derivatives in Machine Learning', 6149],
 ['RMSProp in neural network optimization', 6404]]

In [195]:
make_single_heatmap(families_64_c['Gradient Descent Variants and Applications'], model_64_c)

In [71]:
def make_big_heatmap(family_names, families, model, save = None, display ='cooc'):
    all_ids = []
    all_names = []

    for parent in family_names:
        children = [child[0] for child in families[parent].children]
        ids = [child[1] for child in families[parent].children]
        densities = [model.norms[id] for id in ids]
        
        ids = list(np.array(ids)[np.argsort(densities)])
        ids.append(model.clean_labels[parent]['index'])
        all_ids.append(ids)
        all_names.append(children + [parent])
    
    all_ids = [ind for sublist in all_ids for ind in sublist]
    
    # create matrix, normalized by the minimum of the norms of the two features
    if display == 'cooc':
        min_matrix = np.minimum.outer(model.norms, model.norms)
        norm_mat = (model.mat / min_matrix)[all_ids][:, all_ids]
    else:
        min_matrix = 1
        norm_mat = (model.actsims / min_matrix)[all_ids][:, all_ids]
    np.fill_diagonal(norm_mat, 1)

    heatmap = go.Heatmap(
        z=norm_mat,
        hoverongaps=False,
        hovertemplate='Name: %{x} <br> %{y} <br> %{z}'
    )

    # Create the figure
    fig = go.Figure(data=[heatmap])

    # Update layout for better readability and show only parent labels
    fig.update_layout(
        width=480,
        height=400,
        margin=dict(l=20, r=20, t=20, b=20),
    )
    fig.update_yaxes(showticklabels=False)
    fig.update_xaxes(showticklabels=False)
    fig.show()
    
    if save: fig.write_image(save)
    


In [72]:
make_big_heatmap(['Recurrent Neural Networks (RNNs)', 'Explainability in machine learning and AI', 'Recommendation Systems', 'Deep learning in vision applications', 'Dimensionality Reduction Techniques', ], families_64_c, model_64_c, save = './cooc_grid_cs.png')

In [77]:
make_big_heatmap(['Machine learning','Gamma-Ray Bursts (GRBs)', 'Black holes', 'Periodicity detection methods', 'Gas dynamics'], families_64_a, model_64_a, display = 'cooc', save = 'cooc_grid_astro.png')

# feature family interpretability

In [2]:
def create_radar_chart(feature_f1, overall_f1, feature_names, showlegend=False):
    # Add the first feature name to the end to close the polygon
    categories = feature_names + [feature_names[0]]
    feature_f1 = feature_f1 + [feature_f1[0]]
    
    return [
        go.Scatterpolar(
            r=feature_f1,
            theta=categories,
            fill='toself',
            name='Individual Features',
            line=dict(color='rgb(31, 119, 180)', width=5),
            showlegend=showlegend
        ),
        go.Scatterpolar(
            r=[overall_f1] * len(categories),
            theta=categories,
            name='Family F1 (base)',
            line=dict(color='red', width=5, dash='dash'),
            showlegend=showlegend
        ),
    ]

def plot_radar_charts(all_results, mode = 'f1'):
    n_cols = 2
    n_rows = 2

    fig = make_subplots(
        rows=n_rows, cols=n_cols, 
        specs=[[{'type': 'polar'}] * n_cols] * n_rows,
        subplot_titles=list(all_results.keys()),
        vertical_spacing=0.15
    )

    for i, family in enumerate(all_results):
        row = i // n_cols + 1
        col = i % n_cols + 1
        
        family_results = all_results[family]
        radar_traces = create_radar_chart(family_results['feature_{}'.format(mode)], family_results['family_{}'.format(mode)], family_results['feature_names'], showlegend=(i==0))
        
        for trace in radar_traces:
            fig.add_trace(trace, row=row, col=col)

    # Update layout
    fig.update_layout(
        height=2000,  # Increased height to accommodate larger fonts
        width=2700,   # Increased width for better spacing
        font=dict(family="Palatino", size=12),  # Base font size
        paper_bgcolor='white',
        plot_bgcolor='white',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.12,
            xanchor="right",
            x=1,
            font=dict(size=36)  # Legend font size
        )
    )

    # Update polar axes to ensure consistent scaling and increase label size
    fig.update_polars(
        radialaxis=dict(
            range=[0, 1],
            tickfont=dict(size=12),
            tickangle=45
        ),
        angularaxis=dict(
            tickfont=dict(size=17)
        )
    )

    # Adjust subplot titles position and font size
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=36, family="Palatino")
        i['y'] = i['y'] + 0.05  # Move title up further

    # Save figure

    fig.show()
    fig.write_image('radar_charts.pdf')


In [3]:
all_results = json.load(open('../saerch/sae_data_astroPH/family_analysis_16_3072.json'))

In [4]:
all_results_by_id = {x['superfeature']: x for x in all_results}

In [5]:
all_results_by_id = {x['superfeature']: x for x in all_results}
keys = ['Astrophysical phenomena and interactions', 'Cosmological Mapping Techniques', 'Gravitational wave detection and analysis', 'High-Energy Astrophysical Phenomena'] # 2 bad, 2 good
plot_results = {key: all_results_by_id[key] for key in keys}

In [6]:

plot_radar_charts(plot_results)

In [74]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_radar_chart(feature_f1, overall_f1, feature_names, showlegend=True):
    # Add the first feature name to the end to close the polygon
    categories = feature_names + [feature_names[0]]
    feature_f1 = feature_f1 + [feature_f1[0]]
    
    return [
        go.Scatterpolar(
            r=feature_f1,
            theta=categories,
            fill='toself',
            name='Individual Features',
            line=dict(color='rgb(31, 119, 180)', width=5),
            showlegend=False, #showlegend
        ),
        go.Scatterpolar(
            r=[overall_f1] * len(categories),
            theta=categories,
            name='Family F1 (base)',
            line=dict(color='red', width=5, dash='dash'),
            showlegend=False, #showlegend
        ),
    ]

def plot_single_radar_chart(family_data, mode='f1', base_font_size=12):
    family_name = list(family_data.keys())[0].replace('*', '') 
    family_results = family_data[family_name]

    fig = make_subplots(
        rows=1, cols=1,
        specs=[[{'type': 'polar'}]],
        subplot_titles=[family_name],
    )

    feature_names = [x.replace('*', '') for x in family_results['feature_names']]

    feature_names = ['Numerical relativity in binary systems', 
                 'Third-generation gravitational wave detectors', 
                 'NANOGrav signal interpretations', 
                 'Stochastic processes', 
                 'Gravitational wave polarization modes', 
                 'Deep learning in gravitational wave detection', 
                 'Gravitational Waves', 
                 'Gravitational waves as standard sirens', 
                 'Gravitational wave detection technology', 
                 'Gravitational-wave noise reduction', 
                 'TianQin project', 
                 'KAGRA gravitational-wave observatory', 
                 'Gravitational waves in EMRIs and pulsars', 
                 'GW150914 event and LIGO observations', 
                 'Continuous phenomena', 
                 'Gravitational-wave memory effect', 
                 'Gravitational wave follow-up observations', 
                 'Gravitational waves detection methods']

    radar_traces = create_radar_chart(
        family_results['feature_{}'.format(mode)],
        family_results['family_{}'.format(mode)],
        #family_results['feature_names'],
        feature_names,
        showlegend=True
    )

    print(feature_names)

    for trace in radar_traces:
        fig.add_trace(trace)

    # Update layout
    fig.update_layout(
        height=1200,
        width=2550,
        font=dict(family="Palatino", size=base_font_size),
        paper_bgcolor='white',
        plot_bgcolor='white',
        showlegend=True,
        # legend=dict(
        #     orientation="h",
        #     yanchor="bottom",
        #     y=1.12,
        #     xanchor="right",
        #     x=1,
        #     font=dict(size=base_font_size * 3)
        # )
    )

    # Update polar axes
    fig.update_polars(
        radialaxis=dict(
            range=[0, 1],
            tickfont=dict(size=base_font_size),
            tickangle=45
        ),
        angularaxis=dict(
            tickfont=dict(size=int(base_font_size * 2.5))
        )
    )

    # Adjust subplot title position and font size
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=base_font_size * 3, family="Palatino")
        i['y'] = i['y'] + 0.05  # Move title up further

    # Show and save figure
    fig.show()
    fig.write_image('../output/single_radar_chart.pdf')

In [75]:
all_results = json.load(open('../saerch/sae_data_astroPH/family_analysis_16_3072.json'))

In [76]:
all_results_by_id = {x['superfeature']: x for x in all_results}
keys = ['Gravitational wave detection and analysis'] 
plot_results = {key: all_results_by_id[key] for key in keys}

plot_single_radar_chart(plot_results, base_font_size=14)

['Numerical relativity in binary systems', 'Third-generation gravitational wave detectors', 'NANOGrav signal interpretations', 'Stochastic processes', 'Gravitational wave polarization modes', 'Deep learning in gravitational wave detection', 'Gravitational Waves', 'Gravitational waves as standard sirens', 'Gravitational wave detection technology', 'Gravitational-wave noise reduction', 'TianQin project', 'KAGRA gravitational-wave observatory', 'Gravitational waves in EMRIs and pulsars', 'GW150914 event and LIGO observations', 'Continuous phenomena', 'Gravitational-wave memory effect', 'Gravitational wave follow-up observations', 'Gravitational waves detection methods']
